In [1]:
import sys
import os
sys.path.append("../")
DATA_DIR = "../dataset/training/"

In [2]:
from scripts.preprocessing import *
from scripts.augmentation import *

CREATE AUGMENTED DATASET

In [3]:
ORIGIN_IMAGES = DATA_DIR + "images/"
ORIGIN_MASKS = DATA_DIR + "groundtruth/"
AUG_IMAGES = DATA_DIR + "augmented_images/"
AUG_MASKS = DATA_DIR + "augmented_groundtruth/"
FILES = sorted(os.listdir(ORIGIN_IMAGES))

In [4]:
# Load original dataset
images = [load_image(ORIGIN_IMAGES + FILES[i]) for i in range(len(FILES))]
masks = [load_image(ORIGIN_MASKS + FILES[i]) for i in range(len(FILES))]
print(len(images), len(masks))

100 100


In [5]:
if not os.path.isdir(AUG_IMAGES):
    os.mkdir(AUG_IMAGES)
if not os.path.isdir(AUG_MASKS):
    os.mkdir(AUG_MASKS)

In [6]:
# Generates the 700 new images. The 100 first images are the original images
data_augmented(images, masks, AUG_IMAGES, AUG_MASKS)

/home/yann/.local/lib/python3.10/site-packages/imgaug/imgaug.py:106: DeprecationWarning: Got a float array as the segmentation map in SegmentationMapsOnImage. That is deprecated. Please provide instead a (H,W,[C]) array of dtype bool_, int or uint, where C denotes the segmentation map index.
  warn(msg, category=DeprecationWarning, stacklevel=stacklevel)


NameError: name 'img_float_to_uint8' is not defined